# Coursework 1: ML basics and fully-connected networks

#### Instructions

Please submit a version of this notebook containing your answers on CATe as *CW1*. Write your answers in the cells below each question.

We recommend that you work on the Ubuntu workstations in the lab. This assignment and all code were only tested to work on these machines. In particular, we cannot guarantee compatibility with Windows machines and cannot promise support if you choose to work on a Windows machine.

You can work from home and use the lab workstations via ssh (for list of machines: https://www.doc.ic.ac.uk/csg/facilities/lab/workstations). 

Once logged in, run the following commands in the terminal to set up a Python environment with all the packages you will need.

    export PYTHONUSERBASE=/vol/bitbucket/nuric/pypi
    export PATH=/vol/bitbucket/nuric/pypi/bin:$PATH

Add the above lines to your `.bashrc` to have these enviroment variables set automatically each time you open your bash terminal.

Any code that you submit will be expected to run in this environment. Marks will be deducted for code that fails to run.

Run `jupyter-notebook` in the coursework directory to launch Jupyter notebook in your default browser.

DO NOT attempt to create a virtualenv in your home folder as you will likely exceed your file quota.

**DEADLINE: 7pm, Tuesday 5th February, 2019**

## Part 1

1. Describe two practical methods used to estimate a supervised learning model's performance on unseen data. Which strategy is most commonly used in most deep learning applications, and why?
2. Suppose that you have reason to believe that your multi-layer fully-connected neural network is overfitting. List four things that you could try to improve generalization performance.

\**ANSWERS FOR PART 1 IN THIS CELL*\*
1. (a). Cross-validation: In k-fold cross-validation, we divide the data into k subsets of equal size. We build models k times, each time leaving out one of the subsets from training and use it as the test set.
(b). Hold-out method: divide the dataset to be training set, validation set and testing set. Hold-out is most commonly used. As deep networks always needs long time to train, cross-validation would be too time comsuming.
2. Regularization, noisy inputs, early stopping, reducing layers of networks

## Part 2

1. Why can gradient-based learning be difficult when using the sigmoid or hyperbolic tangent functions as hidden unit activation functions in deep, fully-connected neural networks?
2. Why is the issue that arises in the previous question less of an issue when using such functions as output unit activation functions, provided that an appropriate loss function is used?
3. What would happen if you initialize all the weights to zero in a multi-layer fully-connected neural network and attempt to train your model using gradient descent? What would happen if you did the same thing for a logistic regression model?

\**ANSWERS FOR PART 2 IN THIS CELL*\*
1. Saturated neurons will have near 0 gradients; Multiplications of gradients in deep networks will make gradients vanishing for first several layers; Expensive computation.
2. The gradients are directly applied to the output layer, and an appropriate loss function can be chosen to make the gradients in a proper range, which is not near 0.
3. Each node in same layer will have same weights in neural network after whatever times of training iteration. Logistic regression will work normally.

## Part 3

In this part, you will use PyTorch to implement and train a multinomial logistic regression model to classify MNIST digits.

Restrictions:
* You must use (but not modify) the code provided in `utils.py`. **This file is deliberately not documented**; read it carefully as you will need to understand what it does to complete the tasks.
* You are NOT allowed to use the `torch.nn` module.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultinomialLogisticRegressionClassifier` class below by filling in the missing parts (expected behaviour is prescribed in the documentation):
    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 92% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.


In [1]:
from utils import *

Files already downloaded


In [2]:
# *CODE FOR PART 3.1 IN THIS CELL*

class MultinomialLogisticRegressionClassifier:
    def __init__(self, weight_init_sd=100.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.weight_init_sd = weight_init_sd

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################

        dis=torch.distributions.normal.Normal(0.0,self.weight_init_sd)
        self.w=torch.autograd.Variable(dis.sample((784,10)), requires_grad=True)
        self.b=torch.autograd.Variable(dis.sample((1,10)),requires_grad=True)


        


        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be a Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        batch_size=inputs.shape[0]
        re = torch.matmul(inputs.view(batch_size,784),self.w)+self.b

        return torch.softmax(re,dim=1)

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################

        return [self.w,self.b]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        params_list = []
        for param in self.parameters():
            params_list.append(param.view(-1))
        torch_params = torch.cat(params_list)
            
        return (torch.abs(torch_params).sum())
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        params_list = []    
        for param in self.parameters():
            params_list.append(param.view(-1))
        torch_params = torch.cat(params_list)
            
        return (torch.sqrt(torch.pow(torch_params, 2).sum()))
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

In [3]:
# *CODE FOR PART 3.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
model = MultinomialLogisticRegressionClassifier(weight_init_sd=1e-3)
res = run_experiment(
    model,
    optimizer=optim.Adamax(model.parameters(), 0.01),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=0.0,
    l2_penalty_coef=0.0,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: -0.8375, Accuracy: 0.8789
Validation set:	Average loss: -0.8851, Accuracy: 0.9070

Epoch 1: training...
Train set:	Average loss: -0.8957, Accuracy: 0.9143
Validation set:	Average loss: -0.9003, Accuracy: 0.9143

Epoch 2: training...
Train set:	Average loss: -0.9065, Accuracy: 0.9203
Validation set:	Average loss: -0.9055, Accuracy: 0.9165

Epoch 3: training...
Train set:	Average loss: -0.9119, Accuracy: 0.9235
Validation set:	Average loss: -0.9110, Accuracy: 0.9205

Epoch 4: training...
Train set:	Average loss: -0.9156, Accuracy: 0.9263
Validation set:	Average loss: -0.9117, Accuracy: 0.9195

Epoch 5: training...
Train set:	Average loss: -0.9181, Accuracy: 0.9283
Validation set:	Average loss: -0.9152, Accuracy: 0.9228

Epoch 6: training...
Train set:	Average loss: -0.9201, Accuracy: 0.9295
Validation set:	Average loss: -0.9163, Accuracy: 0.9242

Epoch 7: training...
Train set:	Average loss: -0.9222, Accuracy: 0.9318
Validation set:	Average l

## Part 4

In this part, you will use PyTorch to implement and train a multi-layer fully-connected neural network to classify MNIST digits.

Your network must have three hidden layers with 128, 64, and 32 hidden units respectively.

The same restrictions as in Part 3 apply.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultilayerClassifier` class below by filling in the missing parts of the following methods (expected behaviour is prescribed in the documentation):

    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 97% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.

3. Describe an alternative strategy for initializing weights that may perform better than the strategy we have used here.

In [4]:


# *CODE FOR PART 4.1 IN THIS CELL*

class MultilayerClassifier:
    def __init__(self, activation_fun="sigmoid", weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        super().__init__()
        self.activation_fun = activation_fun
        self.weight_init_sd = weight_init_sd

        if self.activation_fun == "relu":
            self.activation = F.relu
        elif self.activation_fun == "sigmoid":
            self.activation = torch.sigmoid
        elif self.activation_fun == "tanh":
            self.activation = torch.tanh
        else:
            raise NotImplementedError()
        
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        dis=torch.distributions.normal.Normal(0.0,self.weight_init_sd)
        self.w1=torch.autograd.Variable(dis.sample((784,128)), requires_grad=True)
        self.b1=torch.autograd.Variable(dis.sample((1,128)),requires_grad=True)
        self.w2=torch.autograd.Variable(dis.sample((128,64)), requires_grad=True)
        self.b2=torch.autograd.Variable(dis.sample((1,64)),requires_grad=True)
        self.w3=torch.autograd.Variable(dis.sample((64,32)), requires_grad=True)
        self.b3=torch.autograd.Variable(dis.sample((1,32)),requires_grad=True)
        self.w4=torch.autograd.Variable(dis.sample((32,10)), requires_grad=True)
        self.b4=torch.autograd.Variable(dis.sample((1,10)),requires_grad=True)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        batch_size=inputs.shape[0]
        x1 = self.activation(torch.matmul(inputs.view(batch_size,784),self.w1)+self.b1)
        x2 = self.activation(torch.matmul(x1,self.w2)+self.b2)
        x3 = self.activation(torch.matmul(x2,self.w3)+self.b3)
        x4 = torch.softmax(torch.matmul(x3,self.w4)+self.b4, dim=1)
        return x4
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return [self.w1, self.b1,self.w2,self.b2,self.w3,self.b3,self.w4,self.b4]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        params_list = []
        for param in self.parameters():
            params_list.append(param.view(-1))
        torch_params = torch.cat(params_list)
        l1=(torch.abs(torch_params).sum())
#         print('l1: ',l1)
            
        return l1
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        params_list = []    
        for param in self.parameters():
            params_list.append(param.view(-1))
        torch_params = torch.cat(params_list)
        l2=(torch.sqrt(torch.pow(torch_params, 2).sum()))
#         print('l2: ', l2)
        return l2
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

In [ ]:

# *CODE FOR PART 4.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *

model = MultilayerClassifier(activation_fun='relu', weight_init_sd=1e-6)
res = run_experiment(
    model,
    optimizer=optim.Adamax(model.parameters(),5e-3),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=1e-5,
    l2_penalty_coef=1e-5,
    suppress_output=False
)

Epoch 0: training...



Multiply the sampled weights by the square root of (2/N), where N is number of input units for that layer.